In [39]:
import openai, json

client = openai.OpenAI()
messages = []


In [40]:
def get_weather(city):
    return "33 degrees celsius"

FUNCTION_MAP = {
    "get_weather": get_weather
}

In [41]:

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

In [42]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                    },
                } for tool_call in message.tool_calls]
            }
        )
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)
            
            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })

        call_ai()
    else:
        messages.append({
            "role": "assistant",
            "content": message.content,
        })
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [43]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user", 
            "content": message
        })
        print(f"User: {message}")
        call_ai()


User: My name is Mijong
AI: Nice to meet you, Mijong! How can I assist you today?
User: What is my name
AI: Your name is Mijong.
User: What is the weather in Spain
Calling function: get_weather with {"city":"Spain"}
Ran get_weather with args {'city': 'Spain'} for a result of 33 degrees celsius
AI: The current weather in Spain is 33 degrees Celsius. Do you need weather information for a specific city in Spain?
User: 
AI: It seems your message got cut off. How can I assist you further?
User: 
AI: It looks like there hasn't been any message from you. If you have any questions or if there's anything you'd like to know, feel free to ask!


In [44]:
messages

[{'role': 'user', 'content': 'My name is Mijong'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Mijong! How can I assist you today?'},
 {'role': 'user', 'content': 'What is my name'},
 {'role': 'assistant', 'content': 'Your name is Mijong.'},
 {'role': 'user', 'content': 'What is the weather in Spain'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_oomvup0Bm0mHvVviviTqT6Vd',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Spain"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_oomvup0Bm0mHvVviviTqT6Vd',
  'name': 'get_weather',
  'content': '33 degrees celsius'},
 {'role': 'assistant',
  'content': 'The current weather in Spain is 33 degrees Celsius. Do you need weather information for a specific city in Spain?'},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': 'It seems your message got cut off. How can I assist you further?'},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'cont

In [ ]:
ChatCompletion(
    id='chatcmpl-CSjyiQrhimRGHcSGqrKhtlbkp9Jel', 
    choices=[
        Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[
            ChatCompletionMessageFunctionToolCall(
                id='call_JZWcB2PBvrN02dj5CrKXVVc0', 
                function=Function(
                    arguments='{"city":"Spain"}', name='get_weather'), type='function')]))], created=1760965960, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=14, prompt_tokens=93, total_tokens=107, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
